In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 6, 9)
init_page = 15
init_offset = 140

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-07-07
page  1
https://index.sindonews.com/index/0/0?t=2018-07-07
https://autotekno.sindonews.com/read/1319809/121/kawasaki-z900rs-bikin-panas-honda-lahirkan-kembali-cbx-1530983118
Insert berita  Kawasaki Z900RS Bikin Panas, Honda Lahirkan Kembali CBX
masuk
https://international.sindonews.com/read/1319807/43/iran-eksekusi-8-orang-terkait-serangan-mematikan-isis-di-teheran-1530981772
Insert berita  Iran Eksekusi 8 Orang Terkait Serangan Mematikan ISIS di Teheran
masuk
https://international.sindonews.com/read/1319806/44/al-shabaab-serang-kementerian-dalam-negeri-somalia-9-tewas-1530981116
Insert berita  Al-Shabaab Serang Kementerian Dalam Negeri Somalia, 9 Tewas
masuk
